In [15]:
#Finding Samples from survival
import pandas as pd
import numpy as np


def seprate_ids():
    
    cancer_name = ['BLCA','CESC', 'COAD', 'HNSC', 'KIRP', 'LGG', 'LIHC', 'LUAD'] 
#     cancer_name = ['BLCA'] 
    for i in cancer_name:
        df_survival = pd.read_csv('../Data/survival/TCGA-'+i+'.survival.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
        df_lncRNA = pd.read_csv('../Data/TCGA_'+i+'_htseq_fpkm_sorted_lncRNA-reduced-37-T.csv')
        key_survival = df_survival['sample']
        key_lncRNA = df_lncRNA['Ensembl_ID']

        key = set(key_lncRNA)&set(key_survival)

        df_survival = df_survival.loc[df_survival['sample'].isin(key)]
        df_lncRNA = df_lncRNA.loc[df_lncRNA['Ensembl_ID'].isin(key)]
        df_survival = df_survival.sort_values(by=['sample'])
        df_lncRNA = df_lncRNA.sort_values(by=['Ensembl_ID'])

        df_lncRNA['E'] = df_survival['_EVENT'].values
        df_lncRNA['T'] = df_survival['_TIME_TO_EVENT'].values

        df_lncRNA.to_csv('../Data/survival/TCGA_'+i+'_htseq_fpkm_sorted_lncRNA-reduced-37-T-with-survival.csv', index=0)
        print(i, 'finish')

if __name__== "__main__":
    seprate_ids()


BLCA finish
CESC finish
COAD finish
HNSC finish
KIRP finish
LGG finish
LIHC finish
LUAD finish


In [2]:
import pandas as pd
import numpy as np

In [5]:
df_lncRNA = pd.read_csv('../data/survival/TCGA_BLCA_htseq_fpkm_sorted_lncRNA-reduced-37-T-with-survival.csv')

df_dead = df_lncRNA.loc[df_lncRNA['E'] == 0]
df_alive = df_lncRNA.loc[df_lncRNA['E'] == 1]

# print('dead sample:', df_dead.shape)
# print('alive sample:', df_alive.shape)

df_dead_h_1 = df_dead.sample(frac=0.5, replace=0, random_state=1)
df_alive_h_1 = df_alive.sample(frac=0.5, replace=0, random_state=1)



df_dead_h_2 = df_dead.loc[~df_dead['Ensembl_ID'].isin(df_dead_h_1['Ensembl_ID'])]
df_alive_h_2 = df_alive.loc[~df_alive['Ensembl_ID'].isin(df_alive_h_1['Ensembl_ID'])]

# print('1st half of dead sample:', df_dead_h_1.shape)
# print('2nd half of dead sample:', df_dead_h_2.shape)

# print('1st half of alive sample:', df_alive_h_1.shape)
# print('2nd half of alive sample:', df_alive_h_2.shape)

training_set = pd.concat([df_dead_h_1, df_alive_h_1], axis=0, join='inner').sort_index()
testing_set = pd.concat([df_dead_h_2, df_alive_h_2], axis=0, join='inner').sort_index()

# print("first half", df_h_1.shape)
# print("2nd half", df_h_2.shape)


dead sample: (237, 40)
alive sample: (192, 40)
1st half of dead sample: (118, 40)
2nd half of dead sample: (119, 40)
1st half of alive sample: (96, 40)
2nd half of alive sample: (96, 40)
first half (214, 40)
2nd half (215, 40)


In [7]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test 
from lifelines import CoxPHFitter

In [15]:
# Script for Kaplan-Meier Analysis developed by Md Abdullah Al Mamun

PATH = '../../lncRNA/DeepCC/Data/survival/'

def survival_analysis():
    
    df_lncRNA = pd.read_csv('../data/survival/TCGA_BLCA_htseq_fpkm_sorted_lncRNA-reduced-37-T-with-survival.csv')

    df_dead = df_lncRNA.loc[df_lncRNA['E'] == 0]
    df_alive = df_lncRNA.loc[df_lncRNA['E'] == 1]

    df_dead_h_1 = df_dead.sample(frac=0.5, replace=0, random_state=1)
    df_alive_h_1 = df_alive.sample(frac=0.5, replace=0, random_state=1)

    df_dead_h_2 = df_dead.loc[~df_dead['Ensembl_ID'].isin(df_dead_h_1['Ensembl_ID'])]
    df_alive_h_2 = df_alive.loc[~df_alive['Ensembl_ID'].isin(df_alive_h_1['Ensembl_ID'])]

    df_training = pd.concat([df_dead_h_1, df_alive_h_1], axis=0, join='inner').sort_index()
    df_testing = pd.concat([df_dead_h_2, df_alive_h_2], axis=0, join='inner').sort_index()
    
    training_set = df_training.iloc[1:,1:]
    
    cph = CoxPHFitter()
    cph.fit(training_set, duration_col='E', event_col='T', show_progress=True)

    cph.print_summary()
#     print(cph.hazards_)

#     kmf = KaplanMeierFitter()
#     df = pd.read_csv(PATH+"TCGA_CESC_htseq_fpkm_sorted_lncRNA-reduced-37-T-with-survival.csv") # returns a Pandas DataFrame
#     df = df.replace(np.nan,0.0)
    
#     T = df['T']
#     E = df['E']
    
#     lncRNAs = df.columns
    
# #     for i in lncRNAs:
#     groups = df[lncRNAs[5]].values
#     ix = (groups >=0.3 )

#     kmf.fit(T[~ix], E[~ix], label='Low')
#     ax = kmf.plot()

#     kmf.fit(T[ix], E[ix], label='High')
#     ax = kmf.plot(ax=ax)


#     results = logrank_test(T[ix], T[~ix], E[ix], E[~ix], alpha=.99)

#     results.print_summary()

if __name__== "__main__":
    survival_analysis()

Iteration 1: norm_delta = 0.61275, step_size = 0.9500, ll = -932.55521, newton_decrement = 19.75818, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.08437, step_size = 0.9500, ll = -913.98211, newton_decrement = 0.50892, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00790, step_size = 0.9500, ll = -913.46620, newton_decrement = 0.00471, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00054, step_size = 1.0000, ll = -913.46144, newton_decrement = 0.00002, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, ll = -913.46142, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence completed after 5 iterations.
<lifelines.CoxPHFitter: fitted with 213 observations, 0 censored>
      duration col = 'E'
         event col = 'T'
number of subjects = 213
  number of events = 213
    log-likelihood = -913.46
  time fit was run = 2019-07-23 11:08:56 UTC

---
                    coef exp(coef)  se(coef)     z    p  -log2(p)  lower 0.95  upp